In [4]:
import numpy as np
import pandas as pd

import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score

In [2]:
trainData=pd.read_csv("../data/train_set.csv", sep="\t")
print("data shape={}".format(trainData.shape))
trainData.head(3)

data shape=(200000, 2)


,label,text
0,2,2967 6758 339 2021 1854 3731 4109 3792 4149 15...
1,11,4464 486 6352 5619 2465 4802 1452 3137 5778 54...
2,3,7346 4068 5074 3747 5681 6093 1777 2226 7354 6...


In [5]:
with open("../models/tfidf-model.pkl", "rb") as f:
    tfidfv=pickle.load(f)

def get_feats(X):
    return tfidfv.transform(X).toarray()

def evaluate(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro')

def train_test(X_train, y_train, X_test, y_test, model):
    model.fit(X_train, y_train)
    y_pred=model.predict(X_test)
    return evaluate(y_test, y_pred)

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [6]:
labelsData=trainData["label"]
featsData=get_feats(trainData["text"].values)

In [7]:
print("labelsData shape={}, featsData shape={}".format(labelsData.shape, featsData.shape))

labelsData shape=(200000,), featsData shape=(200000, 5000)


In [9]:
skf=StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
model = RandomForestClassifier(n_estimators=100)
results=[]
for train_index, test_index in skf.split(featsData, labelsData):
    X_train, y_train = featsData[train_index], labelsData[train_index]
    print("X shape={}, y shape={}".format(X_train.shape, y_train.shape))
    X_test, y_test = featsData[test_index], labelsData[test_index]
    f1=train_test(X_train, y_train, X_test, y_test, model)
    results.append(f1)

X shape=(160000, 5000), y shape=(160000,)
X shape=(160000, 5000), y shape=(160000,)
X shape=(160000, 5000), y shape=(160000,)
X shape=(160000, 5000), y shape=(160000,)
X shape=(160000, 5000), y shape=(160000,)


AttributeError: 'list' object has no attribute 'mean'

In [10]:
print("mean={}, f1={}".format(np.mean(results), results))

mean=0.8370154885117678, f1=[0.8339345059636758, 0.8336630320405153, 0.8363794884804777, 0.8406588993783254, 0.8404415166958447]
